In [4]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import coo_matrix, hstack
from sklearn.preprocessing import LabelEncoder
import re

In [2]:
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)


def view_na_portion(df,col):
    return sum(df[col].isnull())/len(df)


def one_hot_encoding(df,col,pre_fix):
    df = df.copy()
    df[col] = df[col].fillna('NA_NOT_FOUND')
    print('before encoding, shape is: {}'.format(df.shape))
    for each in df[col].unique():
        name = str(each)
        col_name = pre_fix + name.replace(' ','_')
        df[col_name] = 0
        df.loc[df[col]==each,col_name] = 1
    df = df.drop([col],axis=1)
    print('after encoding, shape is: {}'.format(df.shape))
    return df
    

class ordinal_encoder:
    def __init__(self,fillna='NAN_FILL'):
        self.fillna = fillna
    
    def fit(self,sr,NA_VALUE=None,realNA2Nega1=True):
        """
        spycify the value which has already been encoded. make it to -1 after encoding
        """
        self.NA_VALUE=NA_VALUE
        order = sr.fillna(self.fillna).value_counts()
        if self.NA_VALUE is not None:
            order[NA_VALUE] = 9999999

        if realNA2Nega1:
            order[self.fillna] = 99999999
        order = order.sort_values(ascending=True)


        self.mapping = pd.Series(index=order.index.values, data=list(range(1,len(order)+1)))
        if self.NA_VALUE is not None:
            self.mapping[NA_VALUE] = -1

        if realNA2Nega1:
            self.mapping[self.fillna] = -1
    
    def transform(self,sr):
        return sr.fillna(self.fillna).map(self.mapping)

In [3]:
oe = ordinal_encoder()

In [7]:
X_Train = pd.read_pickle(ENV.application_train_cleaned.value)
print('Train shape: {}'.format(X_Train.shape))

X_Test = pd.read_pickle(ENV.application_test_cleaned.value)
print('Test shape: {}'.format(X_Test.shape))

X_pre = pd.read_pickle(ENV.previous_application_cleaned.value)
print('Previous App shape: {}'.format(X_pre.shape))

X_ins = pd.read_pickle(ENV.installments_payments_clean.value)
print('Installment shape: {}'.format(X_ins.shape))

X_pos = pd.read_pickle(ENV.POS_CASH_balance_clean.value)
print('POS CASH shape: {}'.format(X_pos.shape))

X_cc = pd.read_pickle(ENV.credit_card_balance_clean.value)
print('Credit Card shape: {}'.format(X_cc.shape))

Train shape: (307511, 122)
Test shape: (48744, 121)
Previous App shape: (1670214, 37)
Installment shape: (13605401, 8)
POS CASH shape: (10001358, 8)
Credit Card shape: (3840312, 23)


In [5]:
X_pre_ori = pd.read_csv(ENV.previous_application_ori.value)
print('Previous App shape: {}'.format(X_pre_ori.shape))

X_ins_ori = pd.read_csv(ENV.installments_payments_ori.value)
print('Installment shape: {}'.format(X_ins_ori.shape))

X_pos_ori = pd.read_csv(ENV.POS_CASH_balance_ori.value)
print('POS CASH shape: {}'.format(X_pos_ori.shape))

X_cc_ori = pd.read_csv(ENV.credit_card_balance_ori.value)
print('Credit Card shape: {}'.format(X_cc_ori.shape))

Previous App shape: (1670214, 37)
Installment shape: (13605401, 8)
POS CASH shape: (10001358, 8)
Credit Card shape: (3840312, 23)


# View Columns

In [6]:
X_ins_ori.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT'],
      dtype='object')

In [7]:
X = X_ins_ori.copy()

# View category

In [8]:
col = 'NUM_INSTALMENT_VERSION'
X[col].value_counts()

1.0      8485004
0.0      4082498
2.0       620283
3.0       237063
4.0        55274
5.0        48404
6.0        17092
7.0        16771
9.0         8359
8.0         7814
10.0        4637
11.0        4342
13.0        2951
12.0        2863
15.0        1917
14.0        1906
16.0        1283
17.0        1249
18.0         883
19.0         816
20.0         615
21.0         589
22.0         426
23.0         373
24.0         291
25.0         268
26.0         211
27.0         177
29.0         163
28.0         145
          ...   
35.0          46
39.0          42
36.0          36
37.0          33
38.0          29
40.0          19
43.0          19
41.0          18
42.0          18
44.0          13
61.0           8
45.0           8
72.0           7
46.0           5
47.0           4
48.0           3
49.0           3
50.0           3
52.0           3
51.0           2
53.0           1
54.0           1
55.0           1
56.0           1
57.0           1
58.0           1
59.0           1
178.0         

### Process NUM_INSTALMENT_VERSION

In [9]:
col = 'NUM_INSTALMENT_VERSION'
print(view_na_portion(X_ins_ori,col))
print(X_ins_ori[col].describe())

0.0
count    1.360540e+07
mean     8.566373e-01
std      1.035216e+00
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.780000e+02
Name: NUM_INSTALMENT_VERSION, dtype: float64


### Process NUM_INSTALMENT_NUMBER

In [10]:
col = 'NUM_INSTALMENT_NUMBER'
print(view_na_portion(X_ins_ori,col))
print(X_ins_ori[col].describe())

0.0
count    1.360540e+07
mean     1.887090e+01
std      2.666407e+01
min      1.000000e+00
25%      4.000000e+00
50%      8.000000e+00
75%      1.900000e+01
max      2.770000e+02
Name: NUM_INSTALMENT_NUMBER, dtype: float64


### Process DAYS_INSTALMENT

In [11]:
col = 'DAYS_INSTALMENT'
print(view_na_portion(X_ins_ori,col))
print(X_ins_ori[col].describe())

0.0
count    1.360540e+07
mean    -1.042270e+03
std      8.009463e+02
min     -2.922000e+03
25%     -1.654000e+03
50%     -8.180000e+02
75%     -3.610000e+02
max     -1.000000e+00
Name: DAYS_INSTALMENT, dtype: float64


### process DAYS_ENTRY_PAYMENT

In [12]:
col = 'DAYS_ENTRY_PAYMENT'
print(view_na_portion(X_ins_ori,col))
print(X_ins_ori[col].describe())

X[col] = X[col].fillna(X[col].mean())

0.00021351814621266952
count    1.360250e+07
mean    -1.051114e+03
std      8.005859e+02
min     -4.921000e+03
25%     -1.662000e+03
50%     -8.270000e+02
75%     -3.700000e+02
max     -1.000000e+00
Name: DAYS_ENTRY_PAYMENT, dtype: float64


### process AMT_INSTALMENT

In [13]:
col = 'AMT_INSTALMENT'
print(view_na_portion(X_ins_ori,col))
print(X_ins_ori[col].describe())

0.0
count    1.360540e+07
mean     1.705091e+04
std      5.057025e+04
min      0.000000e+00
25%      4.226085e+03
50%      8.884080e+03
75%      1.671021e+04
max      3.771488e+06
Name: AMT_INSTALMENT, dtype: float64


### process AMT_PAYMENT

In [14]:
col = 'AMT_PAYMENT'
print(view_na_portion(X_ins_ori,col))
print(X_ins_ori[col].describe())

X[col] = X[col].fillna(X[col].mean())

0.00021351814621266952
count    1.360250e+07
mean     1.723822e+04
std      5.473578e+04
min      0.000000e+00
25%      3.398265e+03
50%      8.125515e+03
75%      1.610842e+04
max      3.771488e+06
Name: AMT_PAYMENT, dtype: float64


In [15]:
X_sort = X.sort_values(['SK_ID_CURR','DAYS_INSTALMENT'])

# Feature Engineering

### Count NUM of Version per pre SK ID

In [16]:
group_prev = X.groupby(['SK_ID_PREV'])
SK_ID_PREV = []
SK_ID_CURR = []
count_VERSION = []
count_INSTALLMENT = []
DAY_INS_SPAN = []
DAY_ENTRY_SPAN = []
CNT_LATE_PAYMENT = []
CNT_LESS_PAYMENT = []
TOTAL_AMT_INSTALMENT = []
TOTAL_AMT_PAYMENT = []
INSTAL_START_DAY = []
count = 0
for key,df in group_prev:
    SK_ID_PREV.append(key)
    count_VERSION.append(df.NUM_INSTALMENT_VERSION.nunique())
    count_INSTALLMENT.append(len(df))
    DAY_INS_SPAN.append(df.DAYS_INSTALMENT.max() - df.DAYS_INSTALMENT.min())
    DAY_ENTRY_SPAN.append(df.DAYS_ENTRY_PAYMENT.max() - df.DAYS_ENTRY_PAYMENT.min())
    CNT_LATE_PAYMENT.append(sum(df['DAYS_ENTRY_PAYMENT'] > df['DAYS_INSTALMENT']))
    TOTAL_AMT_INSTALMENT.append(sum(df['AMT_INSTALMENT']))
    TOTAL_AMT_PAYMENT.append(sum(df['AMT_PAYMENT']))
    CNT_LESS_PAYMENT.append(sum(df['AMT_INSTALMENT'] > df['AMT_PAYMENT']))
    INSTAL_START_DAY.append(df.DAYS_INSTALMENT.min())
    count += 1
    if count % 10000 == 0:
        print(count)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000


In [17]:
df_installment_new = pd.DataFrame({'SK_ID_PREV':SK_ID_PREV,
                                   'count_VERSION':count_VERSION,
                                   'count_INSTALLMENT':count_INSTALLMENT,
                                   'DAY_INS_SPAN':DAY_INS_SPAN,
                                   'DAY_ENTRY_SPAN':DAY_ENTRY_SPAN,
                                   'CNT_LATE_PAYMENT':CNT_LATE_PAYMENT,
                                   'CNT_LESS_PAYMENT':CNT_LESS_PAYMENT,
                                   'TOTAL_AMT_INSTALMENT':TOTAL_AMT_INSTALMENT,
                                   'TOTAL_AMT_PAYMENT':TOTAL_AMT_PAYMENT,
                                   'INSTAL_START_DAY':INSTAL_START_DAY})

### add FLAG_TOTAL_PAYMENT_LESS_THAN_INSTALMENT

In [18]:
df_installment_new['FLAG_TOTAL_PAYMENT_LESS_THAN_INSTALMENT'] = (df_installment_new['TOTAL_AMT_INSTALMENT'] > df_installment_new['TOTAL_AMT_PAYMENT']).astype('int')

### add SK_ID_CURR

In [19]:
mapping = X_ins_ori[['SK_ID_CURR','SK_ID_PREV']].drop_duplicates().copy()
mapping = mapping.set_index(['SK_ID_PREV'])['SK_ID_CURR']
df_installment_new['SK_ID_CURR'] = df_installment_new['SK_ID_PREV'].map(mapping)

### add owe amount portion

In [20]:
df_installment_new['OWE_PORTION'] = (df_installment_new['TOTAL_AMT_INSTALMENT'] - df_installment_new['TOTAL_AMT_PAYMENT']) / df_installment_new['TOTAL_AMT_INSTALMENT']
df_installment_new['OWE_PORTION'] = df_installment_new['OWE_PORTION'].replace(np.NINF,-9.9e+05)

In [21]:
scan_nan_portion(df_installment_new).sort_values()

SK_ID_PREV                                 0.0
count_VERSION                              0.0
count_INSTALLMENT                          0.0
DAY_INS_SPAN                               0.0
DAY_ENTRY_SPAN                             0.0
CNT_LATE_PAYMENT                           0.0
CNT_LESS_PAYMENT                           0.0
TOTAL_AMT_INSTALMENT                       0.0
TOTAL_AMT_PAYMENT                          0.0
INSTAL_START_DAY                           0.0
FLAG_TOTAL_PAYMENT_LESS_THAN_INSTALMENT    0.0
SK_ID_CURR                                 0.0
OWE_PORTION                                0.0
dtype: float64

# Saving

In [22]:
# df_installment_new.to_pickle('../../data/cleaned_data/installments_payments_rnn.pkl')
df_installment_new.to_pickle(ENV.installments_payment_clean_rnn.value)

In [10]:
df_installment_new = pd.read_pickle(ENV.installments_payment_clean_rnn.value)

In [12]:
df_installment_new['Normed_CNT_LATE_PAYMENT'] = df_installment_new['CNT_LATE_PAYMENT'] /  df_installment_new['count_INSTALLMENT']

df_installment_new['Normed_CNT_LESS_PAYMENT'] = df_installment_new['CNT_LESS_PAYMENT'] /  df_installment_new['count_INSTALLMENT']

df_installment_new['Install_Payment_Rate'] = df_installment_new['TOTAL_AMT_PAYMENT'] / df_installment_new['TOTAL_AMT_INSTALMENT']

df_installment_new['CNT_installment_per_version']

In [ ]:
retain_col = ['Normed_CNT_LATE_PAYMENT',
              'Normed_CNT_LESS_PAYMENT',
              'Install_Payment_Rate',
              'FLAG_TOTAL_PAYMENT_LESS_THAN_INSTALMENT',
              'OWE_PORTION',
              'count_version',
              'SK_ID_CURR']

In [6]:
df_installment_new

,SK_ID_PREV,count_VERSION,count_INSTALLMENT,DAY_INS_SPAN,DAY_ENTRY_SPAN,CNT_LATE_PAYMENT,CNT_LESS_PAYMENT,TOTAL_AMT_INSTALMENT,TOTAL_AMT_PAYMENT,INSTAL_START_DAY,FLAG_TOTAL_PAYMENT_LESS_THAN_INSTALMENT,SK_ID_CURR,OWE_PORTION
0,1000001,2,2,30.0,50.0,0,0,68443.425,68443.425,-268.0,0,158271,0.000000
1,1000002,2,4,90.0,57.0,0,0,37235.565,37235.565,-1600.0,0,101962,0.000000
2,1000003,1,3,60.0,59.0,0,0,14854.050,14854.050,-94.0,0,252457,0.000000
3,1000004,2,7,180.0,186.0,0,0,33523.155,33523.155,-862.0,0,260094,0.000000
4,1000005,1,11,270.0,254.0,2,2,161735.310,147021.705,-1688.0,1,176456,0.090973
5,1000007,1,5,120.0,133.0,0,0,56234.025,56234.025,-123.0,0,256657,0.000000
6,1000008,2,9,240.0,259.0,0,0,262238.580,262238.580,-1282.0,0,152059,0.000000
7,1000009,1,6,150.0,148.0,0,0,55815.615,55815.615,-457.0,0,343078,0.000000
8,1000010,2,11,300.0,306.0,0,0,1259663.130,1259663.130,-558.0,0,377567,0.000000
9,1000011,1,12,330.0,331.0,2,0,1109158.650,1109158.650,-435.0,0,198678,0.000000
